In [1]:
import os
import urllib.request  
import re
from itertools import chain
import pandas as pd
import numpy as np
import hopsworks
from datetime import datetime, timedelta
import sys

/opt/anaconda3/envs/SMDL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
buoy="46253" # our bouy ID

In [3]:
def get_latest_url(today):
    pred_date = today.strftime("%Y%m%d")

    # There are 4 predictions per day at hours: "00", "06", "12", "18",
    h=int(today.strftime("%H"))
    found = False
    test_url = ""
    attempted_date = today

    while not found:
        pred_hour = "00"
        if h > 5:
            pred_hour = "06"
        if h > 11:
            pred_hour = "12" 
        if h > 17:
            pred_hour = "18"
        test_url = "https://ftpprd.ncep.noaa.gov/data/nccf/com/gfs/prod/gfs." \
        + attempted_date.strftime("%Y%m%d") + \
        "/" + pred_hour + "/wave/station/bulls.t" + pred_hour + "z/gfswave." + buoy + ".bull"
        try:
            urllib.request.urlopen(test_url)
            found = True
        except urllib.error.HTTPError as e: 
            # assume 404, URL not found. Try previous time.
            h = h - 6
            if h < 0:
                attempted_date = attempted_date - timedelta(days=1)
                # if i have to look back >1 day, then just exit with error - because upstream is prob broken
                if (today.day - attempted_date.day > 1):
                    sys.exit("ERROR: Could not download url: " + test_url) 
    print(test_url)
    return test_url, pred_hour

In [4]:
today = datetime.now()
url, pred_hour = get_latest_url(today)

https://ftpprd.ncep.noaa.gov/data/nccf/com/gfs/prod/gfs.20231229/06/wave/station/bulls.t06z/gfswave.46253.bull


In [5]:
pred_hour

'12'

In [5]:
hours = 200 # number of sweel predictions we save
secondary_columns=[]
for i in range(1,hours):
    j=i*2
    secondary_columns.append("height" + str(j))
    secondary_columns.append("period" + str(j))
    secondary_columns.append("direction" + str(j))
    secondary_columns.append("hits_at" + str(j))

secondary_columns

['height2',
 'period2',
 'direction2',
 'hits_at2',
 'height4',
 'period4',
 'direction4',
 'hits_at4',
 'height6',
 'period6',
 'direction6',
 'hits_at6',
 'height8',
 'period8',
 'direction8',
 'hits_at8',
 'height10',
 'period10',
 'direction10',
 'hits_at10',
 'height12',
 'period12',
 'direction12',
 'hits_at12',
 'height14',
 'period14',
 'direction14',
 'hits_at14',
 'height16',
 'period16',
 'direction16',
 'hits_at16',
 'height18',
 'period18',
 'direction18',
 'hits_at18',
 'height20',
 'period20',
 'direction20',
 'hits_at20',
 'height22',
 'period22',
 'direction22',
 'hits_at22',
 'height24',
 'period24',
 'direction24',
 'hits_at24',
 'height26',
 'period26',
 'direction26',
 'hits_at26',
 'height28',
 'period28',
 'direction28',
 'hits_at28',
 'height30',
 'period30',
 'direction30',
 'hits_at30',
 'height32',
 'period32',
 'direction32',
 'hits_at32',
 'height34',
 'period34',
 'direction34',
 'hits_at34',
 'height36',
 'period36',
 'direction36',
 'hits_at36',
 'height

In [6]:
print(len(secondary_columns))

796


In [7]:
buoy="46253" # our bouy ID
def process_url(buoy_url):
    out = []
    for line in urllib.request.urlopen(buoy_url):
        l = line.decode('utf-8') #utf-8 or iso8859-1 or whatever the page encoding scheme is
        row=[]
        #print(l)
        if "Cycle" in l:
            regex = re.findall(r'Cycle.*:\s+([0-9]+)\s+([0-9]+)\s+UTC.*', l)
            if len(regex):
                thedate=regex[0]
        else:
            res = re.match(r'.*[|]\s+([0-9]+)\s+([0-9]+)\s+[|].*', l)
            #waves = re.findall(r'[|]\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9]+)\s+[|]', l)
            waves = re.findall(r'\s+([0-9\.]+)\s+([0-9\.]+)\s+([0-9]+)\s+', l)
            #print("waves", waves)
            if res is not None:
                row.append(thedate)
                row.append(res.groups())
            #print("Waves ",len(waves))
            if len(waves):
                if len(waves) > 3:
                    print("found > 3 waves, reduce to 3")
                    waves = waves[:3]
                b = []
                list(b.extend(item) for item in waves)
                row.append(b)
                my = tuple(chain.from_iterable(row))
                out.append(my)
    return out, thedate

In [8]:
out, thedate =process_url(url)
out


found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 wave

[('20231229', '6', '29', '6', '1.43', '16.9', '89'),
 ('20231229', '6', '29', '7', '1.39', '16.8', '89'),
 ('20231229', '6', '29', '8', '1.34', '16.6', '89'),
 ('20231229', '6', '29', '9', '1.29', '16.5', '89'),
 ('20231229', '6', '29', '10', '1.25', '16.3', '89'),
 ('20231229', '6', '29', '11', '1.21', '16.2', '89'),
 ('20231229', '6', '29', '12', '1.17', '16.1', '89'),
 ('20231229', '6', '29', '13', '1.14', '15.9', '89'),
 ('20231229', '6', '29', '14', '1.11', '15.8', '89'),
 ('20231229', '6', '29', '15', '1.08', '15.8', '89'),
 ('20231229', '6', '29', '16', '1.05', '15.7', '89'),
 ('20231229', '6', '29', '17', '1.02', '15.7', '89'),
 ('20231229', '6', '29', '18', '1.00', '15.6', '89', '0.15', '15.5', '8'),
 ('20231229', '6', '29', '19', '0.97', '15.5', '89', '0.16', '15.5', '10'),
 ('20231229', '6', '29', '20', '0.95', '15.5', '88', '0.17', '15.5', '10'),
 ('20231229', '6', '29', '21', '0.92', '15.4', '88', '0.17', '15.4', '10'),
 ('20231229', '6', '29', '22', '0.90', '15.3', '88'),

In [9]:
pd.DataFrame(out)[:20]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,20231229,6,29,6,1.43,16.9,89,None,None,None,None,None,None
1,20231229,6,29,7,1.39,16.8,89,None,None,None,None,None,None
2,20231229,6,29,8,1.34,16.6,89,None,None,None,None,None,None
3,20231229,6,29,9,1.29,16.5,89,None,None,None,None,None,None
4,20231229,6,29,10,1.25,16.3,89,None,None,None,None,None,None
5,20231229,6,29,11,1.21,16.2,89,None,None,None,None,None,None
6,20231229,6,29,12,1.17,16.1,89,None,None,None,None,None,None
7,20231229,6,29,13,1.14,15.9,89,None,None,None,None,None,None
8,20231229,6,29,14,1.11,15.8,89,None,None,None,None,None,None
9,20231229,6,29,15,1.08,15.8,89,None,None,None,None,None,None


In [14]:
primary_columns=['pred_dtime', 'hour', 'pred_day', 'pred_hour', 'height1', 'period1', 'direction1', 'height2', 
         'period2', 'direction2', 'height3', 'period3', 'direction3'] 

# changed angels to valid swell from around 10 - 65 degrees
def is_valid_swell_direction(direction):
    if int(direction) > 66 or int(direction) < 9:
        return False
    return True

def best_height(row):
    best_secondary=2
    # Check which is best secondary swell - swell 2 or swell 3?
    if row['direction3'] != None:
        if is_valid_swell_direction(row['direction3']):
            if is_valid_swell_direction(row['direction2']) == False :
                best_secondary=3    
    best_direction = "direction" + str(best_secondary)
    best=1
    # Check which is best of swell 1 and secondary swell ?
    if row[best_direction] != None and is_valid_swell_direction(row[best_direction]) == True:
        if is_valid_swell_direction(row['direction1']) == False:
            best=best_secondary
                
    height = row['height' + str(best)]
    period = row['period' + str(best)]
    direction = row['direction' + str(best)]
        
    return pd.Series([height, period, direction])

# feature engineering - estimate the time at which the swell arrives at Lahinch from buoy
def estimate_hits_at(row):
    # baseline estimate
    hits_at = row['pred_dtime'] + row['hour_offset'] + timedelta(hours=0.4) 
    return pd.Series([hits_at])
    

    
today = datetime.now()
url, pred_hour = get_latest_url(today)
print(url)
res,thedate=process_url(url)
df = pd.DataFrame(res, columns=primary_columns)
df['pred_dtime'] = pd.to_datetime(df['pred_dtime'], format='%Y%m%d')

# Generate the timedelta series (40 minutes for each row)
time_offsets = pd.Series([timedelta(minutes=40) * i for i in range(len(df))])

# Create the cumulative sum of time offsets
#cumulative_time_offsets = time_offsets.cumsum()

#df.insert(loc=0, column="hour_offset", value=(df.reset_index().index*0.4))
#df['hour_offset'] = df.hour_offset.astype('timedelta64[m]')
df['hour_offset'] = time_offsets
df['pred_dtime'] = df['pred_dtime'] + df.hour.astype('timedelta64[h]')
df

https://ftpprd.ncep.noaa.gov/data/nccf/com/gfs/prod/gfs.20231229/06/wave/station/bulls.t06z/gfswave.46253.bull
https://ftpprd.ncep.noaa.gov/data/nccf/com/gfs/prod/gfs.20231229/06/wave/station/bulls.t06z/gfswave.46253.bull
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce to 3
found > 3 waves, reduce 

,pred_dtime,hour,pred_day,pred_hour,height1,period1,direction1,height2,period2,direction2,height3,period3,direction3,hour_offset
0,2023-12-29,6,29,6,1.43,16.9,89,None,None,None,None,None,None,0 days 00:00:00
1,2023-12-29,6,29,7,1.39,16.8,89,None,None,None,None,None,None,0 days 00:40:00
2,2023-12-29,6,29,8,1.34,16.6,89,None,None,None,None,None,None,0 days 01:20:00
3,2023-12-29,6,29,9,1.29,16.5,89,None,None,None,None,None,None,0 days 02:00:00
4,2023-12-29,6,29,10,1.25,16.3,89,None,None,None,None,None,None,0 days 02:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2023-12-29,6,14,2,0.19,2.5,332,None,None,None,None,None,None,10 days 08:40:00
374,2023-12-29,6,14,3,0.16,2.5,333,0.15,4.2,49,None,None,None,10 days 09:20:00
375,2023-12-29,6,14,4,0.16,4.2,44,None,None,None,None,None,None,10 days 10:00:00
376,2023-12-29,6,14,5,0.16,4.2,44,None,None,None,None,None,None,10 days 10:40:00


In [15]:
df[['height','period','direction']]=df.apply(best_height, axis=1)
df[['hits_at']]=df.apply(estimate_hits_at, axis=1)
df['beach_id'] = 1
df.drop(['height1', 'period1', 'direction1', 'height2', 'period2', 'direction2', 'hour_offset',
          'height3', 'period3', 'direction3','hour', 'pred_day', 'pred_hour'], axis=1, inplace=True) 
df

,pred_dtime,height,period,direction,hits_at,beach_id
0,2023-12-29,1.43,16.9,89,2023-12-29 00:24:00,1
1,2023-12-29,1.39,16.8,89,2023-12-29 01:04:00,1
2,2023-12-29,1.34,16.6,89,2023-12-29 01:44:00,1
3,2023-12-29,1.29,16.5,89,2023-12-29 02:24:00,1
4,2023-12-29,1.25,16.3,89,2023-12-29 03:04:00,1
...,...,...,...,...,...,...
373,2023-12-29,0.19,2.5,332,2024-01-08 09:04:00,1
374,2023-12-29,0.15,4.2,49,2024-01-08 09:44:00,1
375,2023-12-29,0.16,4.2,44,2024-01-08 10:24:00,1
376,2023-12-29,0.16,4.2,44,2024-01-08 11:04:00,1


## Connect to your Hopsworks cluster 

In [12]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/194711
Connected. Call `.close()` to terminate connection gracefully.


In [16]:
version = 1
swells_fg = fs.get_or_create_feature_group(name="buoy_swells_huntington",
                version=version,
                primary_key=["beach_id"],
                event_time="hits_at",
                description="Buoy surf height predictions",
                online_enabled=True,
                statistics_config={"enabled": True, "histograms": True, "correlations": True}
                )
swells_fg.insert(df)


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/194711/fs/194630/fg/363890


Uploading Dataframe: 100.00% |██████████| Rows 378/378 | Elapsed Time: 00:06 | Remaining Time: 00:00


Launching job: buoy_swells_huntington_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/194711/jobs/named/buoy_swells_huntington_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7fd70a0d4640>, None)